This notebook contains Avida resource plotting animation examples and associated loading utilities.  You must have ffmpeg installed in order for the animations to be render to HTML5 video.

*Only single resource experiments are allowed at this time.*

Avida must be compiled and available on the system.

For resource plots, it is recommended that an organism that is not capable of replication is used.  This will speed up the experiment run.  One recommendation would be to add nop-X to the default instruction set; change the h-divide command in the default organism to nop-X.  It is also recommended that all mutations in the avida configuration file (avida.cfg) are set to zero and that death is disabled in the configuration file as well.

In [5]:
# %matlab inline
# Enabling the above will show both the animation movies and the
# last plot in the animation.

import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
import pdb
import pandas
import warnings
import subprocess
import tempfile
import progressbar
import time

from pprint import PrettyPrinter as pprint
from matplotlib import animation, rc
from IPython.display import HTML


#rc('text',usetex=True)  # Use tex for text rendering; allows for inlined math
warnings.filterwarnings('ignore') # Disable warnings for clarity

In [2]:
animation.writers.list()

['ffmpeg', 'ffmpeg_file', 'imagemagick', 'imagemagick_file', 'html']

Demonstration Implementation with Random Data
========================

In [3]:
def init_animation():
    """
    Initialize the animation prior to the first
    frame being drawn.  Really, this does nothing
    in this example.
    """
    ax = sb.heatmap(np.zeros(world))
    plt.title('Test')
    return ax
    

    
def animate(ndx, *fargs):
    """
    Draw a frame in the animation.
    
    :param ndx: Current top-level index into data
    :param fargs: list of parameters passed by the
                    fargs argument in the animate
                    function
    :return: the plot generated
    
    """
    plt.clf()  #Needed to clear the figure
    data = fargs[0][ndx]
    params = fargs[1]
    ax = sb.heatmap(data, cmap=cmap, **params)
    ax.tick_params(axis='both', bottom='off', labelbottom='off',
                   left='off', labelleft='off')
    plt.title(fargs[2])
    return ax
    

def build_animation(data):
    """
    Given a 3-dimension array of data, create an animation.
    
    :param data: the data to animate
    :return: the animation object
    """
    
    # Open our figure that we're going to animate upon
    fig, ax = plt.subplots()

    # To keep the colormap constant, we need to find its bounds
    # before we begin any plotting
    dmax = 0
    for dmap in data:
        m = np.amax(dmap)
        if m > dmax:
            dmax = m
    
    # fig is the figure we're operating on
    # animate is the name of the function used to draw a frame
    # frames is either an iterable or generator for the animation function
    # fargs is the arguments passed as a list of additional positional args
    #    to the animate function; here we're using it to pass the data
    #    directly as an argument as the first element, the arguments to
    #    pass as the kw dictionary to the plot function, and the third
    #    argument is the title of the chart.
    # interval is measured in milliseconds and is the time a frame shown
    # 
    # We're relying on the example in the cell below to set some globals
    # for controlling things like world size, the interval, and the color
    # map used for the heatmap.
    anim = animation.FuncAnimation(fig, animate, 
                                frames=range(0,len(data)),
                                init_func=init_animation,
                               fargs=[data, {'vmin':0, 'vmax':dmax}, 'test'], 
                               interval=interval)
    return anim


   
    
    

In [4]:
# To be used with the demonstration functions only
world = (60,60)  # Our world size
interval = 200   # ms per frame
cmap = sb.color_palette('YlGn')  # Seaborn colormap for the heatmap
data = np.random.rand(10,100,100)  # Generate our random data to animate
HTML(build_animation(data).to_html5_video())  # Build & embed animation

# Note: if the HTML function call is inside of a loop, wrap it with a
# call to display() in order for it to generate.

Avida-Specific Resource Plotting
================================

The methods and examples below will work directly with avida.

The first section shows how to load already generated data from a directory.  The second section shows how to call avida from here and grab the data generated.  Both end up animating the results.

The action PrintSpatialResources will generate data compatible with these utilities.

In [6]:
def gen_resource_grid(data, world_size):
    """
    This is a generator that passes the update and the cell-grid's
    resources.  The generator will run until there are no additional
    updates to plot.  *HOWEVER*, a limitation in the animation library
    requires that the number of frames in the animation be known in
    advanced.  See build_resource_animation's notes.
    
    :param data: A pandas data frame that contains the update of a
                 resource in the first column, and columns 3-end
                 contains the abundance of a resource per cell.
    :yield: A tuple of the update and the grid of resources per cell
    """
    ndx = 0
    while ndx < len(data):
        yield ndx, data.iloc[ndx][0], data.iloc[ndx, 2:].reshape(world_size).astype('float')
        ndx = ndx + 1
    

def animate_resource(info, *fargs):
    """
    Animate a single frame of a resource grid.
    
    :param info: The tuple from the generator; first value is the 
                 current frame; the second value is the update;
                 and the third value is the resouce grid to plot
    :param *fargs:  Additional positional parameters as a list as
                    specified by the FuncAnimation call by the fargs
                    argument.  In this case, fargs is a list of 
                    four values: the parameters to pass the heatmap
                    drawing function, the title of the plot, 
                    the colormap to be used in the heatmap, and
                    the progress bar to show how the animation is
                    coming along.
    :return: The plot created for the current frame.
    """
    ndx, update, grid = info  # From our generator
    params = fargs[0]    # The keyword parameters to pass the heatmap
    title = fargs[1]     # The title of the plot
    cmap = fargs[2]      # The colormap for the plot
    pbar = fargs[3]      # The progress bar
    plt.clf()  # Clear our figure
    ax = sb.heatmap(grid, cmap=cmap, cbar_kws={'label': 'Abundancd $log_{10}$'}, **params)
    ax.tick_params(axis='both', bottom='off', labelbottom='off',
                   left='off', labelleft='off')  # Turn off ticks
    plt.title(title)
    plt.xlabel('Update {}'.format(update))
    pbar.update(ndx)
    return ax
    


def build_resource_animation(pdata, world_size, title, interval, cmap):
    """
    Animate resources from an Avida run using the PrintSpatialResources
    action.
    
    :param pdata: A Pandas dataframe containing the information from the
                  spatial resource output file.
    :param world_size: A tuple containing the (Y,X) values of the world
    :param title: The title to include with the plots in the animation
    :param interval: The animation speed in ms
    :param cmap: The seaborn colormap to be used to generate the resource
                 heatmap
    :return: The animation object for the resource plot
    """
    fig, ax = plt.subplots()
    
    dmax = np.log10(pdata.iloc[:,2:]).max().max()  #Maximum abundance value
    nframes = len(pdata)  #Number of frames to animate
    
    # Progress bar setup
    # The progress bar will show how much time remains along with
    # a filling bar to indicate that the animation is being built.
    # The pbar_widgets establishes how teh progress bar will be
    # formatted.  pbar is the progress bar object itself, which
    # is being passed to our animate function to update.
    pbar_widgets = [progressbar.FormatLabel('Building Animation'),
                    '  ',
                    progressbar.Bar(), 
                    progressbar.Percentage(),
                    '  ',
                    progressbar.ETA(
                        format_zero = '%(elapsed)s elapsed',
                        format_not_started='', 
                        format_finished='',
                        format_NA='',
                        format='%(eta)s remaining')]
    pbar = progressbar.ProgressBar(widgets=pbar_widgets, max_value=nframes-1).start()
    
    anim = animation.FuncAnimation(fig, animate_resource, frames=gen_resource_grid(pdata, world_size),
                                   fargs=[{'vmin':0, 'vmax':dmax}, title, cmap, pbar], 
                                   interval=interval, save_count=nframes
                                  )
    return anim
                                 

Example with Fixed Folders
------------------------
The section of code below iterates over a series of data directories inside of a single folder, animating and displaying each of them.

In [7]:
world = (60,60)  # world size in Y, X
interval = 50    # animation speed in ms
cmap = sb.color_palette('YlGn')  # Heatmap colormap
data_dir_root = 'avida' # Where are all the data directories?
data_dirs = ['data']    # Which data directories should we use?
filename = 'resources.dat'  # What is the name of the 
                            # spatial resource file output in each 
                            # data directory


# Iterate over each data directory
# Load the spatial resource data from each directory
# Display the animation
# Note that the display(...) call is needed if the HTML
# function call is located inside of a loop.
for dir in data_dirs:
    path = '{}/{}/{}'.format(data_dir_root, dir, filename)
    data = pandas.read_csv(path, comment='#', skip_blank_lines=True, 
                           delimiter=' ',header=None)
    display(HTML(build_resource_animation(
        data, world, 'Resources', interval, cmap).to_html5_video()))
    

Building Animation  |####################################|100%  0:00:26 elapsed

Functions to Plot Dynamic Experiment
----------------------------------
The code below will allow the user to setup an Avida experiment in the Python code, run the experiment, and animate the resource grid when the experiment is completed.

In [13]:
def finished_widget(code):
    '''
    Just a utility function to return a progressbar widget
    that labels whether a process exited with errors or not
    '''
    if code == 0:
        return progressbar.FormatLabel('[OK]')
    else:
        return progressbar.FormatLabel('[FAILED]')

    
    
    
def run_process(args, cwd, title):
    '''
    Runs the shell command args in the directory cwd with a
    spinny progress bar.
    
    :param args:  The shell command to run.
    :param cwd:   The directory to execute the command in
    :param title: Title for the spinny bar
    
    :return: None or raises an exception if the process exits
             with a non-zero exit code.  If an exception is
             thrown, print the stdout/err from the subprocess
             as well.
    '''
    
    # subprocess.PIPE can only hold about 64k worth of data before
    # it hangs the chid subprocess.  To get around this, we're writing
    # the standard output and standard error to this temporary file.
    tmp_stdout = tempfile.NamedTemporaryFile(mode='w', delete=False)
    file_stdout = open(tmp_stdout.name, 'w')
    
    # Spawn the child subprocess.  We're using Popen so we can animate
    # our spinning progressbar widget.  We're using the shell=True to
    # deal with issues trying to spawn avida properly with the command
    # line options constructed as a single string.
    proc = subprocess.Popen(args,
                   cwd=cwd, 
                   shell=True,
                   stdout=file_stdout, 
                   stderr=subprocess.STDOUT,
                   universal_newlines=True)
    
    # Set up our progressbar spinny wheel.
    pbar_widgets = [progressbar.FormatLabel(title),
                    '  ',
                    progressbar.AnimatedMarker(), 
                    '  ',
                    progressbar.FormatLabel(''),
                    '  ',
                    progressbar.ETA(
                        format_zero = '',
                        format_not_started='', 
                        format_finished='%(elapsed)s elapsed',
                        format_NA='',
                        format='')]
    pbar = progressbar.ProgressBar(widgets=pbar_widgets).start()

    # Wait for our process to finish; poll() will return the exit
    # code when it's done or None if it is still running.  The wheel
    # spins via the update().
    while proc.poll() is None:
        time.sleep(0.25)
        pbar.update()
    return_code = proc.wait()  # Grab our subprocess return code
    file_stdout.close()        # Close our subprocess's output streams file
    
    # Rest our widget to be in its final state
    pbar_widgets[2] = ' ' # Delete the spinny wheel
    pbar_widgets[4] = finished_widget(return_code)  # Describe in the pbar how we exited
    pbar.finish()  # Finish the progress bar
    
    # Handle issues if the process failed.  
    # Print the standard output / error from the process temporary
    # file out, then raise a CalledProcessError exception.
    if return_code != 0:
        with open(tmp_stdout.name) as file_stdout:
            print(file_stdout.read())
        raise subprocess.CalledProcessError(return_code, args)



def write_temp_file(contents):
    """
    Write contents to a temporary file and return the file's path.
    
    :param contents: The contents to write into the temp file
    :return: The path to the file.
    """
    f = tempfile.NamedTemporaryFile(mode='w', delete=False)
    f.write(contents)
    n = f.name
    f.close()
    return n


def run_experiment(cfg):
    """
    Run an avida experiment and return a Pandas dataframe containing
    the resource abundances per cell over the course of the experiment.
    
    The only configuration files that is generated is the environment
    file.  All other configuration settings are found in the "avida"
    folder local to the directory of this notebook.  The data directory
    and the environment file are stored on the system as temporary files.
    
    :param cfg: a dictionary containing configuration settings to set.
                Keys:
                    args   Arguments to include on the command line
                    environment  Contents of the environment file
                    events       Contents of the events file
                If the file contents are not specified, the defaults
                in the avida root directory are used.
    :return: The Pandas dataframe containing the resource ata
    """
    
    cwd = cfg['cwd']    # Where is the avida working directory?
    args = cfg['args']  # Begin building our avida argument list
    
    # If we need to build a new environment file, make it
    if 'environment' in cfg:
        path = write_temp_file(cfg['environment'])
        args = args + ' -set ENVIRONMENT_FILE {}'.format(path)

    # If we need to build a new events file, make it
    if 'events' in cfg:
        path = write_temp_file(cfg['events'])
        args = args + ' -set EVENT_FILE {}'.format(path)

    # Create a temporary directory to hold our avida output
    data_dir = tempfile.TemporaryDirectory()
    args = args + ' -set DATA_DIR {}'.format(data_dir.name)

    # Run avida
    run_process('./avida ' + args, cwd, 'Running Avida')
    
    # Load and return our spatial resource data
    res_path = '{}/resources.dat'.format(data_dir.name)
    data = pandas.read_csv(res_path, comment='#', skip_blank_lines=True,
                       delimiter=' ', header=None)
    return data


def plot_experiment(cfg):
    """
    Run and plot the resource abundances per cell as an animation.
    
    :param cfg:  Configuration dictionary; it is also passed to
                 the run_experiment function
                 Keys used here:
                    title   Title of the plots in the animation
    :return: The animation object of the resources over time
    """
    world = (60,60)  # The Y,X size of the world
    interval = 50    # The speed of the animation in ms
    cmap = sb.color_palette('YlGn')  # The colormap of the heatmap
    data = run_experiment(cfg)  # Run the experiment and grab the data
    title = cfg['title']
    return build_resource_animation(data, world, 
                                    title, interval, cmap)

def create_resource_video(cfg):
    """
    Run the experiment, animate the resource data, and return a video.
    """
    return plot_experiment(cfg).to_html5_video()

Dynamic Experiments & Plots
-------------

We can define a dictionary that defines how to configure an Avida experiment and animate the spatial resource abudance information that is produced.  *This is currently restricted to a single resource*.

See the notes in the run_experiment function for additional details.

In [14]:
# Define a dictionary with the configuration settings
inflow_outflow = {
     'cwd':'avida',
     'args':'-s 10',  # Command line arguments
     
    'environment':
     r"""
     RESOURCE food:geometry=torus:initial=0:inflow=1
     """,  # The environment file contents
    
    'events': 
    r"""
    u begin Inject default-heads.org
    u 0:25:end PrintSpatialResources resources.dat
    u 5000 exit
    """,  # The events file
    
    'title':'Inflow/Outflow using grid',  # The title of the plot
}

# Run, plot, and display animation
HTML(create_resource_video(inflow_outflow))

Running Avida     [OK]  0:00:01 elapsed                                        
Building Animation  |####################################|100%  0:00:57 elapsed

In [ ]:
"""
Gradient resource prototype test
"""
inflow_outflow = {
     'cwd':'avida',
     'args':'-s 10',  # Command line arguments
     
    'environment':
     r"""
     GRADIENT_RESOURCE resORN0:height=15:plateau=100:spread=25:common=1:updatestep=1000000:peakx=47:peaky=12:plateau_inflow=100:initial=100
     """,  # The environment file contents
    
    'events': 
    r"""
    u begin Inject default-heads.org
    u 0:25:end PrintSpatialResources resources.dat
    u 5000 exit
    """,  # The events file
    
    'title':'Inflow/Outflow using grid',  # The title of the plot
}

# Run, plot, and display animation
HTML(create_resource_video(inflow_outflow))

Running Avida     [OK]  0:00:01 elapsed                                        
Building Animation  |#######                           | 21%  0:00:43 remaining